# Notebook 6: Detecção de Anomalias com AutoEncoders (Deep Learning)

## Objetivo
Usar Redes Neurais para aprender padrões complexos e não-lineares.

## O Conceito
Um AutoEncoder é uma rede neural que aprende a copiar a entrada para a saída, passando por um 'gargalo' (camada latente comprimida).
- Ele aprende a comprimir e descomprimir dados 'normais' perfeitamente.
- Quando ele tenta processar uma anomalia, ele falha em reconstruí-la corretamente (erro alto).

In [ ]:
!pip install -q pyod pandas matplotlib seaborn tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import generate_data

# TensorFlow é verboso
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Geração de Dados
Vamos usar um dataset com bastante dados para justificar o uso de Deep Learning.

In [ ]:
n_train = 5000
n_test = 1000
n_features = 20 # 20 colunas de dados simluados
contamination = 0.1

X_train, X_test, y_train, y_test = generate_data(
    n_train=n_train, 
    n_test=n_test, 
    n_features=n_features, 
    contamination=contamination,
    random_state=42
)

print("Treino shape:", X_train.shape)

## 2. Construindo o AutoEncoder
Definimos a arquitetura da rede (número de neurônios nas camadas ocultas).

In [ ]:
# hidden_neurons: lista com a qtd de neuronios em cada camada da rede encoder/decoder
# Ex: [20 -> 10 -> 5 -> 10 -> 20]
# epochs: Quantas vezes passar pelos dados
# batch_size: Quantos exemplos processar por vez

clf_ae = AutoEncoder(
    hidden_neurons=[15, 10, 5, 10, 15], 
    epochs=30, 
    batch_size=32,
    contamination=contamination,
    verbose=0
)

print("Treinando AutoEncoder... (Isso pode demorar alguns segundos)")
clf_ae.fit(X_train)

## 3. Avaliação
Assim como no PCA, a métrica é o erro de reconstrução.

In [ ]:
y_test_pred = clf_ae.predict(X_test)
y_test_scores = clf_ae.decision_function(X_test)

from pyod.utils.data import evaluate_print
evaluate_print('AutoEncoder', y_test, y_test_scores)

### Comparando Pontos Normais vs Anômalos
Vamos plotar o erro de reconstrução.

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(y_test_scores[y_test==0], bins=50, alpha=0.5, label='Normal', color='blue')
plt.hist(y_test_scores[y_test==1], bins=50, alpha=0.5, label='Anomalia', color='red')
plt.xlabel('Erro de Reconstrução')
plt.title('Distribuição do Erro de Reconstrução')
plt.legend()
plt.show()

## Conclusão
O histograma mostra claramente que anomalias (vermelho) têm erros de reconstrução ordens de magnitude maiores que os dados normais (azul). Essa separação clara torna o AutoEncoder muito poderoso para dados complexos onde métodos lineares (PCA) ou de distância simples (KNN) falhariam.